# Imports

In [76]:
from numpy.random import seed
seed(888)
from tensorflow import set_random_seed
set_random_seed(404)

In [77]:
import os 
import numpy as np 
import tensorflow as tf
from time import strftime
from PIL import Image

In [78]:
 print(tf.__version__)

1.14.0


# Constants

In [79]:
X_TEST_PATH ='original\digit_xtest.csv'
X_TRAIN_PATH = 'original\digit_xtrain.csv'
Y_TRAIN_PATH = 'original\digit_ytrain.csv'
Y_TEST_PATH = 'original\digit_ytest.csv'

NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_WIDTH*IMAGE_HEIGHT*CHANNELS

LOGGING_PATH = 'tensorboard_mnist_digit_log/'

# Data

In [80]:
y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)
y_test_all = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)
x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)
x_test_all = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)

In [81]:
y_train_all.shape

(60000,)

# Explore

In [82]:
x_train_all.shape

(60000, 784)

# Data Preprocessing

## Re-scale



In [83]:
x_train_all, x_test_all = x_train_all/255.0, x_test_all/255.0 

In [84]:
y_train_all = np.eye(10)[y_train_all]
y_test_all = np.eye(10)[y_test_all]


# Creating validation dataset from training data

In [85]:
x_val = x_test_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]

In [86]:
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

# Setup Tensorflow Graph

In [87]:
X = tf.placeholder(tf.float32, shape=[None, TOTAL_INPUTS])
Y = tf.placeholder(tf.float32, shape=[None, NR_CLASSES])

### Neural Network Architecture

#### Hyperparameters

In [88]:
nr_epochs = 50
learning_rate = 0.001

n_hidden1 = 512
n_hidden2 = 64


#### Layer 1

In [89]:

# with tf.name_scope('hidden_layer1'):

#     initial_w1 = tf.truncated_normal(shape=[TOTAL_INPUTS, n_hidden1], stddev=0.01, seed=42)
#     w1 = tf.Variable(initial_value=initial_w1)

#     initial_b1 = tf.constant(value = 0.0, shape=[n_hidden1])
#     b1 = tf.Variable(initial_value = initial_b1)

#     layer1_in = tf.matmul(X, w1) + b1
#     layer1_out = tf.nn.relu(layer1_in)

#### Layer 2

In [90]:

# with tf.name_scope('hidden_layer2'):
#     initial_w2 = tf.truncated_normal(shape=[n_hidden1, n_hidden2], stddev=0.01, seed=42)

#     w2 = tf.Variable(initial_value=initial_w2)
#     initial_b2 = tf.constant(value = 0.0, shape=[n_hidden2])
#     b2 = tf.Variable(initial_value = initial_b2)

#     layer2_in = tf.matmul(layer1_out, w2) + b2

#     layer2_out = tf.nn.relu(layer2_in)

#### Output Layer

In [91]:
# with tf.name_scope('output_layer'):

#     initial_w3 = tf.truncated_normal(shape=[ n_hidden2, NR_CLASSES], stddev=0.01, seed=42)
#     w3 = tf.Variable(initial_value=initial_w3)

#     initial_b3 = tf.constant(value = 0.0, shape=[NR_CLASSES])
#     b3 = tf.Variable(initial_value = initial_b3)

#     layer3_in = tf.matmul(layer2_out, w3) + b3

#     output = tf.nn.softmax(layer3_in)

In [92]:
def setup_layer(input, weight_dim, bias_dim, name):
    
    with tf.name_scope(name):
        initial_w = tf.truncated_normal(shape=weight_dim, stddev=0.1, seed=42)
        w = tf.Variable(initial_value=initial_w, name='W')

        initial_b = tf.constant(value=0.0, shape=bias_dim)
        b = tf.Variable(initial_value=initial_b, name='B')

        layer_in = tf.matmul(input, w) + b
        
        if name=='out':
            layer_out = tf.nn.softmax(layer_in)
        else:
            layer_out = tf.nn.relu(layer_in)
        
        tf.summary.histogram('weights', w)
        tf.summary.histogram('biases', b)

        
        
        return layer_out

In [93]:
layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1], 
                      bias_dim=[n_hidden1], name='layer_1')

layer_drop = tf.nn.dropout(layer_1, keep_prob = 0.8, name = 'dropout_layer')

                    

layer_2 = setup_layer(layer_drop, weight_dim=[n_hidden1, n_hidden2], 
                      bias_dim=[n_hidden2], name='layer_2')

output = setup_layer(layer_2, weight_dim=[n_hidden2, NR_CLASSES], 
                      bias_dim=[NR_CLASSES], name='out')

model_name = f'{n_hidden1}-DO-{n_hidden2} LR{learning_rate} E{nr_epochs}'


# Tensorboard

In [94]:
# Folder for Tensorboard

folder_name = f'Model {strftime}'
directory = os.path.join(LOGGING_PATH, folder_name)

try:
    os.makedirs(directory)
except OSError as exception:
    print(exception.strerror)
else:
    print('Successfully created directories!')

Cannot create a file when that file already exists


## Loss, Optimization & Metrics

#### Loss Function

In [95]:
with tf.name_scope('loss_calc'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = Y, logits=output))

#### Optimizer: Adam

In [96]:
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_step  = optimizer.minimize(loss)

#### Accuracy Metrics

In [97]:
with tf.name_scope('accuracy_calc'):
    model_prediction = tf.argmax(output, axis = 1, name = 'prediction')
    correct_pred = tf.equal(tf.argmax(output, axis = 1), tf.argmax(Y, axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [98]:
with tf.name_scope('performance'):
    tf.summary.scalar('accuracy', accuracy)
    tf.summary.scalar('cost', loss)


### Check Input Images in Tensorboard

In [99]:
with tf.name_scope('show_image'):
    x_image = tf.reshape(X, [-1, 28, 28, 1])

    tf.summary.image('image_input', x_image, max_outputs = 4)

## Run Session

In [100]:
sess = tf.Session()

#### Setup Filewriter and Merge Summaries

In [101]:
merged_summary = tf.summary.merge_all()

train_writer = tf.summary.FileWriter(directory + '/train')

train_writer.add_graph(sess.graph)

validation_writer = tf.summary.FileWriter(directory + '/validation')


In [102]:
init = tf.global_variables_initializer()
sess.run(init)

In [103]:
size_of_batch = 1000


In [104]:
num_examples = y_train.shape[0]
nr_iterations = int(num_examples/size_of_batch)

index_in_epochs = 0

In [105]:
def next_batch(batch_size, data, labels):
    global num_examples
    global index_in_epochs

    start = index_in_epochs
    index_in_epochs+=batch_size

    if index_in_epochs > num_examples:
        start = 0
        index_in_epochs = batch_size

    end = index_in_epochs
    return data[start:end], labels[start:end]


### Train Loop

In [106]:
#=============Training Dataset====================
for epoch in range(nr_epochs):
    for i in range(nr_iterations):
        batch_x, batch_y = next_batch(batch_size = size_of_batch, data = x_train, labels = y_train)

        feed_dictionary = {X:batch_x, Y:batch_y}

        sess.run(train_step, feed_dict=feed_dictionary)

        s, batch_accuracy = sess.run(fetches=[merged_summary, accuracy], feed_dict = feed_dictionary)


    train_writer.add_summary(s, epoch)

    print(f'Epoch {epoch}\t| Training Accuracy = {batch_accuracy}')

#=======================Validation===============

    summary = sess.run(fetches=merged_summary, feed_dict={X:x_val, Y:y_val})
    validation_writer.add_summary(summary, epoch)
print("Training is Over!")

Epoch 0	| Training Accuracy = 0.8600000143051147
Epoch 1	| Training Accuracy = 0.8809999823570251
Epoch 2	| Training Accuracy = 0.8870000243186951
Epoch 3	| Training Accuracy = 0.9620000123977661
Epoch 4	| Training Accuracy = 0.9679999947547913
Epoch 5	| Training Accuracy = 0.9750000238418579
Epoch 6	| Training Accuracy = 0.9779999852180481
Epoch 7	| Training Accuracy = 0.9789999723434448
Epoch 8	| Training Accuracy = 0.9810000061988831
Epoch 9	| Training Accuracy = 0.9810000061988831
Epoch 10	| Training Accuracy = 0.9819999933242798
Epoch 11	| Training Accuracy = 0.9860000014305115
Epoch 12	| Training Accuracy = 0.984000027179718
Epoch 13	| Training Accuracy = 0.9890000224113464
Epoch 14	| Training Accuracy = 0.9900000095367432
Epoch 15	| Training Accuracy = 0.9879999756813049
Epoch 16	| Training Accuracy = 0.9890000224113464
Epoch 17	| Training Accuracy = 0.9900000095367432
Epoch 18	| Training Accuracy = 0.9890000224113464
Epoch 19	| Training Accuracy = 0.9890000224113464
Epoch 20	| 

### Saving the Model

In [107]:
outputs = {'accuracy/prediction':model_prediction}
inputs = {'X':X}

In [108]:
tf.compat.v1.saved_model.simple_save(sess, 'SavedModel', inputs, outputs)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: SavedModel\saved_model.pb


### Prediction Testing

In [109]:
img = Image.open('original/test_img.png')
img

In [110]:
img_array = np.invert(img.convert('L'))

In [111]:
test_img = img_array.ravel()

In [112]:
pred = sess.run(fetches = tf.argmax(output, axis=1), feed_dict = {X: [test_img]})

In [113]:
print(pred)

[2]


### Reset for next run

In [114]:
train_writer.close()
validation_writer.close()
sess.close()
tf.reset_default_graph()